In [6]:
import pandas as pd

# CSV 파일 읽기
df_target = pd.read_csv("표준코드/산지코드_직팜.csv", encoding='cp949')
df_source = pd.read_csv("기상/산지코드_직팜_new.csv", encoding='cp949')

# 위도/경도가 없을 경우를 대비해 컬럼이 없으면 생성
if '위도' not in df_target.columns:
    df_target['위도'] = None
if '경도' not in df_target.columns:
    df_target['경도'] = None

# 중복 제거 후 참조 딕셔너리 생성
coord_dict = df_source.drop_duplicates(subset='직팜산지이름') \
                      .set_index('직팜산지이름')[['위도', '경도']] \
                      .to_dict('index')

# 반복문으로 값 채우기 (이미 채워진 경우는 건너뜀)
for idx, row in df_target.iterrows():
    # 이미 위도/경도가 있으면 스킵
    if pd.notnull(row['위도']) and pd.notnull(row['경도']):
        continue

    name = row['직팜산지이름']
    if name in coord_dict:
        df_target.at[idx, '위도'] = coord_dict[name]['위도']
        df_target.at[idx, '경도'] = coord_dict[name]['경도']

# 결과 저장
df_target.to_csv("기상/산지코드_직팜_위경도_병합완료.csv", index=False, encoding='cp949')